# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [6]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.nn.functional as F
from torch.autograd import Variable

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# MNIST Dataset
train_dataset = torchvision.datasets.MNIST(root='./data/',train=True,transform=transforms.ToTensor(),download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',train=False,transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=BATCH_SIZE,shuffle=False)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 输入1通道，输出10通道，kernel 5*5
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        # fully connect
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        # in_size = 64
        in_size = x.size(0) # one batch
        # x: 64*10*12*12
        x = F.relu(self.mp(self.conv1(x)))
        # x: 64*20*4*4
        x = F.relu(self.mp(self.conv2(x)))
        # x: 64*320
        x = x.view(in_size, -1) # flatten the tensor
        # x: 64*10
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

Next, we can start to train and evaluate!

In [4]:
def train(epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def loss():
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

for epoch in range(1, 10):
    train(epoch)
    loss()

/home/yangjinyi2000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.317359
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.606252
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.450804


/home/yangjinyi2000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/yangjinyi2000/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.3230, Accuracy: 8993/10000 (90%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.241951
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.198641
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.284072

Test set: Average loss: 0.1770, Accuracy: 9468/10000 (95%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.131441
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.098740
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.173051

Test set: Average loss: 0.1321, Accuracy: 9604/10000 (96%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.150855
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.256261
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.165370

Test set: Average loss: 0.1086, Accuracy: 9687/10000 (97%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.145346
Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.148958
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.132051

Test set: Average loss: 0.0944, Accuracy: 9714/10000 (97%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.053519
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.121649
Train Epoch: 6 [5120